In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [2]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data = np.array(data)
m,n = data.shape
# m is the amount of examples we have and n is the number of labels - 1 (exclude the first one)
np.random.shuffle(data)

data_dev = data[0:1000].T
# we wanted our data as transpose matrix of the original in mathematical model

Y_dev = data_dev[0]
X_dev = data_dev[1:n]

#  edits
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.

In [4]:
X_dev.shape

(784, 1000)

In [5]:
#  now the first thing to do is to initialize the parameters
#     to get values in between -0.5 to 0.5
def init_params():
    w1 = np.random.randn(10,784) - 0.5
    w2 = np.random.randn(10,10) - 0.5
    b1 = np.random.randn(10,1) - 0.5
    b2 = np.random.randn(10,1) - 0.5
    return w1,w2,b1,b2

def ReLU(x):
    return np.maximum(0,x)
# this will return elementwise ReLU function for this input

def softmax(x):
    a = np.exp(x)/ sum(np.exp(x))
    return a

def forward_prop(w1,w2,b1,b2,X):
    z1 = w1.dot(X) + b1
    a1 = ReLU(z1)
    z2 = w2.dot(a1) + b2
    a2 = softmax(z2)
    return z1,a1,z2,a2
    
def one_hot(Y):
    onehoty = np.zeros((Y.size,Y.max()+1))
    onehoty[np.arange(Y.size),Y] =1
    onehoty = onehoty.T
    return onehoty

def deriv_ReLU(z):
    return z>0

# now start with the new variables
def back_prop(z1,a1,z2,a2,w2,x,y):
    onehoty = one_hot(y)
    dz2 = a2 - onehoty
    dw2 = (1/m) * dz2.dot(a1.T)
    db2 = 1/m * np.sum(dz2)
    dz1 = w2.T.dot(dz2)*deriv_ReLU(z1)
    dw1 = 1/m * dz1.dot(x.T)
    db1 = 1/m * np.sum(dz1)
    return dw1,db1,dw2,db2

def update_params(w1,w2,b1,b2,dw1,db1,dw2,db2,alpha):
    w1 = w1 -alpha*dw1
    b1 = b1 - alpha*db1
    w2 = w2 - alpha*dw2
    b2 = b2 - alpha*db2
    return w1,b1,w2,b2

def get_predictions(a2):
    return np.argmax(a2,0)
def get_accuracy(predictions,y):
    print(predictions,y)
    return np.sum(predictions == y)/ y.size

In [6]:
def gradient_descent(x,y,iterations,alpha):
    w1,w2,b1,b2 = init_params()
    for i in range(iterations):
        z1,a1,z2,a2 = forward_prop(w1,w2,b1,b2,x)
        dw1 ,db1,dw2,db2 = back_prop(z1,a1,z2,a2,w2,x,y)
        w1,b1,w2,b2 = update_params(w1,w2,b1,b2,dw1,db1,dw2,db2,alpha)
        if i%50 == 0:
            print("iteration: " , i)
            print("Accuracy: ", get_accuracy(get_predictions(a2),y))
    return w1,b1,w2,b2

In [7]:
w1,b1,w2,b2 = gradient_descent(X_train,Y_train,100,0.1)

iteration:  0
[0 0 0 ... 0 0 0] [0 1 1 ... 8 5 8]
Accuracy:  0.0981219512195122
iteration:  50
[0 0 0 ... 0 0 0] [0 1 1 ... 8 5 8]
Accuracy:  0.0981219512195122
